In [3]:
#!pip install geopy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

#현 경로를 파라미터로 지정
SRC_PATH = os.path.join(os.getcwd(), "src")
os.chdir(SRC_PATH)

In [4]:
#데이터 로드
child_df = pd.read_csv('김해시_어린이집_정상.csv', encoding = 'cp949')
kinder_df = pd.read_csv('김해시유치원정보.csv', encoding = 'cp949')
kinder_car_df = pd.read_csv('김해시_유치원_통학차량현황.csv', encoding = 'cp949')
hospital_df = pd.read_csv('김해시_소아청소년과_현황.csv', encoding = 'cp949')
hospital_df = hospital_df[['병원명', '주소']]

#유치원정보와 통합차량현황 병합
kinder_df_united = kinder_df.merge(kinder_car_df, how = 'inner', on = '유치원명')

#병합된 데이터프레임 컬럼 재조정
kinder_df_united = kinder_df_united[['유치원명','설립유형_x','주소','운영시간', '만3세학급수', '만4세학급수',
       '만5세학급수', '혼합학급수', '특수학급수', '만3세유아수', '만4세유아수', '만5세유아수', '혼합유아수',
       '특수유아수','인가총정원수', '3세모집정원수', '4세모집정원수','5세모집정원수', '혼합모집정원수', '특수학급모집정원수',
       '차량운영여부', '운행차량수', '신고차량수', '9인승신고차량수','12인승신고차량수', '15인승신고차량수']]


#경위도를 추가한 새로운 데이터프레임 생성
from pandas.io.json import json_normalize
import json
import requests


for df in [kinder_df_united, hospital_df]:
    df1 = df.copy()
    x_list = []
    y_list = []
    for addr in df1.loc[:,'주소']:
    
        #NCP Console에서 복사한 Client ID와 Client Secret 값
        client_id = 'd31sws195a'
        client_secret = 'dIeeJAxWQNuD0N05SxUFNCkEbgIgc66oovZwI8wf'

        endpoint = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
        url = f'{endpoint}?query={addr}'

        #header
        headers = {
                    "X-NCP-APIGW-API-KEY-ID": client_id,
                    "X-NCP-APIGW-API-KEY": client_secret,
                    }

        # 요청
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            if len(res.json()['addresses']) != 0:
                try :
                    json_str = json.loads(res.text)
                    address_xy = json_str['addresses'][0]
                    address_x = address_xy['x']
                    address_y = address_xy['y']
                except:
                    address_x = np.nan
                    address_y = np.nan
            else :
                address_x = np.nan
                address_y = np.nan
        else:
            address_x, address_y = "ERROR[" + str(res.status_code) + "]"
        
        x_list.append(address_x)
        y_list.append(address_y)
    df1['x'] = x_list
    df1['y'] = y_list
    globals()[f'{df.columns[0]}_output'] = df1



In [5]:
#경위도 변환이 되지 않은 값 발생
display(유치원명_output.isnull().sum())
print('\n')
display(병원명_output.isnull().sum())
display(유치원명_output[유치원명_output['x'].isnull()])
display(병원명_output[병원명_output['x'].isnull()])

유치원명          0
설립유형_x        0
주소            0
운영시간          0
만3세학급수       57
만4세학급수       34
만5세학급수       24
혼합학급수         0
특수학급수        81
만3세유아수       57
만4세유아수       34
만5세유아수       24
혼합유아수         0
특수유아수        81
인가총정원수        0
3세모집정원수      57
4세모집정원수      34
5세모집정원수      24
혼합모집정원수       0
특수학급모집정원수    81
차량운영여부        0
운행차량수         0
신고차량수         0
9인승신고차량수      0
12인승신고차량수     0
15인승신고차량수     0
x             1
y             1
dtype: int64

병원명    0
주소     0
x      5
y      5
dtype: int64

,유치원명,설립유형_x,주소,운영시간,만3세학급수,만4세학급수,만5세학급수,혼합학급수,특수학급수,만3세유아수,...,혼합모집정원수,특수학급모집정원수,차량운영여부,운행차량수,신고차량수,9인승신고차량수,12인승신고차량수,15인승신고차량수,x,y
32,수인사유치원,사립(사인),경상남도 김해시 분성로3번길 181-33,09시00분~17시20분,4.0,4.0,4.0,0,NaN,93.0,...,0,NaN,Y,4,4,0,0,4,NaN,NaN


,병원명,주소,x,y
37,이신호의원,경상남도 김해시 진영읍 진영로 211 211,NaN,NaN
41,이봉환소아청소년과의원,경상남도 김해시 진영읍 김해대로361번길 16 16,NaN,NaN
80,부경신경과의원,경상남도 김해시 진례면 송현로 12 12,NaN,NaN
120,(재)대한불교관음종대동의원,경상남도 김해시 대동면 동남로85번길 48 48,NaN,NaN
121,한사랑병원,경상남도 김해시 김해대로2272번길 43-47 (강동),NaN,NaN


In [6]:
 #주소 - 좌표 변환함수
def xygenerator(addr):    
    client_id = 'd31sws195a'
    client_secret = 'dIeeJAxWQNuD0N05SxUFNCkEbgIgc66oovZwI8wf'

    endpoint = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    url = f'{endpoint}?query={addr}'

    #header
    headers = {
               "X-NCP-APIGW-API-KEY-ID": client_id,
               "X-NCP-APIGW-API-KEY": client_secret,
                }

    # 요청
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        if len(res.json()['addresses']) != 0:
            try :
                json_str = json.loads(res.text)
                address_xy = json_str['addresses'][0]
                address_x = address_xy['x']
                address_y = address_xy['y']
            except:
                address_x = np.nan
                address_y = np.nan
        else :
            address_x = np.nan
            address_y = np.nan
    else:
        address_x, address_y = "ERROR[" + str(res.status_code) + "]"
    
    return address_x, address_y

'''경위도 변환되지 않은 주소 전처리'''
#경상남도 김해시 분성로 3번길 181-33 : 경상남도 김해시 내동 270
#경상남도 김해시 진영읍 진영로 211 211 : 경상남도 김해시 진영읍 진영로 211
#경상남도 김해시 진영읍 김해대로361번길 16 16 : 경상남도 김해시 진영읍 김해대로361번길 16
#경상남도 김해시 진례면 송현로 12 12 : 경상남도 김해시 진례면 송현로 12
#경상남도 김해시 대동면 동남로85번길 48 48 : 경상남도 김해시 대동면 동남로85번길 48
#경상남도 김해시 김해대로2272번길 43-47 (강동) : 경상남도 김해시 강동로 65

#전처리된 주소 좌표 변환
for addr in ['경상남도 김해시 내동 270',
             '경상남도 김해시 진영읍 진영로 211',
             '경상남도 김해시 진영읍 김해대로361번길 16',
             '경상남도 김해시 진례면 송현로 12',
             '경상남도 김해시 대동면 동남로85번길 48',
             '경상남도 김해시 강동로 65'] : 
    print(xygenerator(addr))

('128.8601738', '35.2462949')
('128.7362535', '35.3027291')
('128.7298955', '35.3082271')
('128.7494911', '35.2487787')
('128.9913235', '35.2378524')
('128.8764710', '35.2199847')


In [7]:
xygenerator('경남 김해시 삼계동 1545')

('128.8656349', '35.2747797')

In [5]:
#경위도 변환되지 않은 주소를 전처리 후, 재변환된 값 치환

#경상남도 김해시 분성로 3번길 181-33 : 경상남도 김해시 내동 270
유치원명_output['x'].iloc[32] = 128.8601738
유치원명_output['y'].iloc[32] = 35.2462949

#경상남도 김해시 진영읍 진영로 211 211 : 경상남도 김해시 진영읍 진영로 211
#경상남도 김해시 진영읍 김해대로361번길 16 16 : 경상남도 김해시 진영읍 김해대로361번길 16
#경상남도 김해시 진례면 송현로 12 12 : 경상남도 김해시 진례면 송현로 12
#경상남도 김해시 대동면 동남로85번길 48 48 : 경상남도 김해시 대동면 동남로85번길 48
#경상남도 김해시 김해대로2272번길 43-47 (강동) : 경상남도 김해시 강동로 65
병원명_output['x'].iloc[37] = 128.7362535
병원명_output['y'].iloc[37] = 35.3027291
병원명_output['x'].iloc[41] = 128.7298955
병원명_output['y'].iloc[41] = 35.3082271
병원명_output['x'].iloc[80] = 128.7494911
병원명_output['y'].iloc[80] = 35.2487787
병원명_output['x'].iloc[120] = 128.9913235
병원명_output['y'].iloc[120] = 35.2378524
병원명_output['x'].iloc[121] = 128.8764710
병원명_output['y'].iloc[121] = 35.2199847


C:\Users\user\AppData\Local\Temp\ipykernel_328140\3626129562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  유치원명_output['x'].iloc[32] = 128.8601738
C:\Users\user\AppData\Local\Temp\ipykernel_328140\3626129562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  유치원명_output['y'].iloc[32] = 35.2462949


In [11]:
유치원명_output.to_csv('김해시_유치원정보_차량유무_경위도포함.csv', encoding = 'cp949')
병원명_output.to_csv('김해시_소아청소년과_현황_경위도포함.csv', encoding = 'cp949')